# Advanced Lane Finding Project:

## Import Dependencies

In [1]:
import cv2
import glob

## Calibrate Camera

In [ ]:
def load_calibration_images():
    paths = glob.glob('camera_cal/*.jpg')
    return np.array([cv2.imread(path) for path in paths])


def calibrate_cam(images):
    pass